In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

In [7]:

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)


In [20]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_cordinates",
        "description": "Get altitude and longitude for provided city name.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string"} 
              },
            "required": ["city"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

In [21]:
completion=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "What're the altitued and longitude in Seoul today?"}]
)

In [22]:
content=completion.choices[0].message.content
print(content)

Seoul, South Korea, is located at an approximate latitude of 37.5665° N and a longitude of 126.9780° E. These coordinates remain relatively constant over time, so you can rely on them regardless of the specific date.
